# 1 - Criando o ambiente e importando as bibliotecas

In [ ]:
# Importação da biblioteca pandas
import pandas as pd

In [ ]:
# Instalação dos requisitos para o PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# Configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
import findspark
findspark.init()

In [ ]:
# iniciar uma sessão local
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Introducao").getOrCreate()

In [ ]:
# Verifica o SparkContext
print(spark)

# Exibe a Spark version
print(spark.version)

3.1.1


In [ ]:
# Definir quais funcões sql pyspark vamos importar
from pyspark.sql.functions import *

#2 - Criando DataFrames

In [ ]:
# Em Pandas
'''
Nesta apostila um objetivo com sufixo pd será um objeto pandas, por
exemplo dfpd
'''
nome = ['Douglas', 'Daniela', 'Pedro', 'Maria', 'Eduardo', 'Ester']
idade = [45, 7, 65, 64, 42, 37]
altura = [1.85, 1.23, 1.75, 1.67, 1.82, 1.73]
peso = [70, 22, 87, 64, 96, 68]
sexo = ['M', 'F', 'M', 'F', 'M', 'F']

dfpd = pd.DataFrame({'nome': nome, 'idade': idade, 'altura': altura,
                     'peso': peso, 'sexo': sexo})
display(dfpd)

In [ ]:
# Em ambinete PySpark nosso df se chamará dfps
data = [("Douglas", 45, 1.85, 70, "M"),
        ("Daniela", 7, 1.23, 22, "F"),
        ("Pedro", 65, 1.75, 87, "M"),
        ("Maria", 64, 1.67, 64, "F"),
        ("Eduardo", 42, 1.82, 96, "M"),
        ("Ester", 37, 1.73, 68, "F")]
columns = ["nome", "idade", "altura", "peso", "sexo"]
dfps = spark.createDataFrame(data, columns)
dfps.show()

#3 - Exibindo linhas e colunas

In [ ]:
#Pandas
dfpd.shape

(6, 5)

In [ ]:
# PySpark
print(f"Quantidade de linhas e colunas: {dfps.count()}, {len(dfps.columns)}")

Quantidade de linhas e colunas: 6, 5


#4 - Exibindo aleatoriamente uma linha DF

In [ ]:
# Pandas
dfpd.sample()

,nome,idade,altura,peso,sexo
3,Maria,64,1.67,64,F


In [ ]:
# PySpark
dfps.sample(False, 0.9999999999).show(1) # (probabilidade de vazio, chance de ser sorteado novamente a mesma linha)


+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Douglas|   45|  1.85|  70|   M|
+-------+-----+------+----+----+
only showing top 1 row



#5 - Exibindo Informações sobre os Tipos de Cada Atributo

In [ ]:
# Pandas
dfpd.dtypes

In [ ]:
# Pandas
dfpd.info()

In [ ]:
# PySpark
dfps.printSchema()

#6 - Exibindo Informações do Índice das Linhas

In [ ]:
# Pandas
dfpd.index

RangeIndex(start=0, stop=6, step=1)

In [ ]:
# PySpark
dfps.rdd.map(lambda row: row).zipWithIndex().toDF().show()

#7 - Estatísticas Descritivas

In [ ]:
# Pandas
medidas = dfpd.describe()
display(medidas)

In [ ]:
# PySpark
dfps.describe().show()

#8 - Exibindo as Primeiras e Últimas Linhas

In [ ]:
# Primeiras Linhas em Pandas
dfpd.head()

In [ ]:
# Últimas linhas em Pandas
dfpd.tail()

In [ ]:
# Primeiras 3 linhas em PySpark
dfps.show(3)

In [ ]:
# Últimas 3 linhas em PySpark
dfps.orderBy(col("nome").desc()).show(3)

#9 - Exibindo Valores de uma Linha pelo Rótulo e Índice

In [ ]:
# Rótulo Pandas
dfpd.loc[0]

,0
nome,Douglas
idade,45
altura,1.85
peso,70
sexo,M


In [ ]:
# Rótulo Pandas
medidas.loc['count']

,count
idade,6.0
altura,6.0
peso,6.0


In [ ]:
# Índice Pandas
dfpd.iloc[0]

,0
nome,Douglas
idade,45
altura,1.85
peso,70
sexo,M


In [ ]:
# Índice Pandas
medidas.iloc[0]

,count
idade,6.0
altura,6.0
peso,6.0


In [ ]:
# Rótulo PySpark
dfps.where(col("sexo") == "F").show()

+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Daniela|    7|  1.23|  22|   F|
|  Maria|   64|  1.67|  64|   F|
|  Ester|   37|  1.73|  68|   F|
+-------+-----+------+----+----+



In [ ]:
# Rótulo PySpark
dfps.where((col("nome") != "Douglas") & (col("peso") >= 68)).show()

+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|  Pedro|   65|  1.75|  87|   M|
|Eduardo|   42|  1.82|  96|   M|
|  Ester|   37|  1.73|  68|   F|
+-------+-----+------+----+----+



In [ ]:
# Ìndice PySpark
'''
Em PySpark não temos suporte direto à índices, pois os dados
não estão alocados em disco, mas podemos fazer ajustes técnicos (gambiarra)
como veremos mais pra frente.
'''
dfps.select(dfps.columns[0]).show()

# 10 - Exibindo Mais de uma Linha pelo Rótulo e Índice

In [ ]:
# Mais de uma linha pelo rótulo Pandas
dfpd.loc[[0, 3, 5]]

,nome,idade,altura,peso,sexo
0,Douglas,45,1.85,70,M
3,Maria,64,1.67,64,F
5,Ester,37,1.73,68,F


In [ ]:
# Mais de uma linha pelo índice Pandas
dfpd.iloc[[0, 3, 5]]

,nome,idade,altura,peso,sexo
0,Douglas,45,1.85,70,M
3,Maria,64,1.67,64,F
5,Ester,37,1.73,68,F


In [ ]:
# Mais de uma linha pelo rótulo PySpark
dfps.filter(col("nome").isin("Douglas", "Maria", "Ester")).show()

+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Douglas|   45|  1.85|  70|   M|
|  Maria|   64|  1.67|  64|   F|
|  Ester|   37|  1.73|  68|   F|
+-------+-----+------+----+----+



In [ ]:
# Mais de uma linha pelo índice PySpark
'''
Na linha a seguir vamos utilizar a função collect do PySpark para
organizar todoo df em linhas em um objeto rdd
'''
linhas = dfps.collect()
'''
Na linha a seguir vamos pegar esse objeto rdd e iterar (compreensão de lista)
sobre ele para selecionar apenas as linhas 0, 3 e 5 do objeto rdd e armazená-las
em um objeto chamado linhas selecionadas
'''
linhas_selecionadas = [linhas[i] for i in [0, 3, 5]]
'''
Na linha a seguir vamos transformar o objeto linhas selecionadas em um df
pyspark chamado df_ps_linhas_selecionadas
'''
df_ps_linhas_selecionadas = spark.createDataFrame(linhas_selecionadas, dfps.schema)
'''
Agora é só exibí-lo
'''
df_ps_linhas_selecionadas.show()

+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
|Douglas|   45|  1.85|  70|   M|
|  Maria|   64|  1.67|  64|   F|
|  Ester|   37|  1.73|  68|   F|
+-------+-----+------+----+----+



# 11 - Fatiando Colunas pelo Rótulo e Índice

In [ ]:
# Rótulo Pandas
dfpd.loc[:, ['nome','peso']]

,nome,peso
0,Douglas,70
1,Daniela,22
2,Pedro,87
3,Maria,64
4,Eduardo,96
5,Ester,68


In [ ]:
# ìndice Pandas
dfpd.iloc[:, [0]]

,nome
0,Douglas
1,Daniela
2,Pedro
3,Maria
4,Eduardo
5,Ester


In [ ]:
# Rótulo PySpark
dfps.select("nome").show()

+-------+
|   nome|
+-------+
|Douglas|
|Daniela|
|  Pedro|
|  Maria|
|Eduardo|
|  Ester|
+-------+



In [ ]:
# Índice PySpark
dfps.select(dfps.columns[0]).show()

+-------+
|   nome|
+-------+
|Douglas|
|Daniela|
|  Pedro|
|  Maria|
|Eduardo|
|  Ester|
+-------+



In [ ]:
# Índice PySpark Múltiplos
selected_columns = [dfps.columns[i] for i in [0, 3]]
dfps.select(*selected_columns).show()

+-------+----+
|   nome|peso|
+-------+----+
|Douglas|  70|
|Daniela|  22|
|  Pedro|  87|
|  Maria|  64|
|Eduardo|  96|
|  Ester|  68|
+-------+----+



# 12 - Exibindo Linhas Específicas de uma Coluna pelo Rótulo e Índice

In [ ]:
# Rótulo Pandas
dfpd.loc[[0, 2, 4], ['nome']]

,nome
0,Douglas
2,Pedro
4,Eduardo


In [ ]:
# Índice Pandas
dfpd.iloc[[0, 2, 4], [0]]

,nome,altura,sexo
0,Douglas,1.85,M
2,Pedro,1.75,M
4,Eduardo,1.82,M


In [ ]:
# Rótulo PySpark
dfps.filter(col("nome").isin("Douglas", "Pedro", "Eduardo")).select("nome").show()

+-------+
|   nome|
+-------+
|Douglas|
|  Pedro|
|Eduardo|
+-------+



#13 - Agrupamentos e Cálculos Estatísticos

In [ ]:
# Pandas
dfpd.groupby('sexo').count()

,nome,idade,altura,peso
sexo,,,,
F,3,3,3,3
M,3,3,3,3


In [ ]:
# Pandas
dfpd[['sexo', 'idade']].groupby('sexo').mean()

,idade
sexo,
F,36.000000
M,50.666667


In [ ]:
# PySpark
dfps.groupBy("sexo").count().show()

+----+-----+
|sexo|count|
+----+-----+
|   F|    3|
|   M|    3|
+----+-----+



In [ ]:
# PySpark
'''
objeto.groupby('coluna_dimensão').agg(tipo_de_agregação('coluna_métrica').alias('nome_a_exibir')).show()
'''
dfps.groupBy("sexo").agg(avg("idade").alias("idade_media")).show()

+----+------------------+
|sexo|       idade_media|
+----+------------------+
|   F|              36.0|
|   M|50.666666666666664|
+----+------------------+



In [ ]:
# PySpark
'''
objeto.groupby('coluna_dimensão').agg(tipo_de_agregação('coluna_métrica').alias('nome_a_exibir')).show()
'''
dfps.groupBy("sexo").agg(min("idade").alias("idade_media")).show()

+----+-----------+
|sexo|idade_media|
+----+-----------+
|   F|          7|
|   M|         42|
+----+-----------+



In [ ]:
# PySpark
'''
objeto.groupby('coluna_dimensão').agg(tipo_de_agregação('coluna_métrica').alias('nome_a_exibir')).show()
'''
dfps.groupBy("sexo").agg(max("idade").alias("idade_media")).show()

+----+-----------+
|sexo|idade_media|
+----+-----------+
|   F|         64|
|   M|         65|
+----+-----------+



# 14 - Importando arquivos

In [ ]:
from google.colab import files
files.upload()

In [ ]:
arquivo = "/content/funcionarios.csv"
flights = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .csv(arquivo)

In [ ]:
#Verificando o shape do pyspark dataframe
print((flights.count(), len(flights.columns)))

(1470, 35)


In [ ]:
# Exibindo as 5 primeiras linhas
flights.show(5)

+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de contratos de 

In [ ]:
# Exibindo o nome das colunas e seus respectivos tipos
flights.printSchema()

In [ ]:
'''
A Criação da view temporária é necessária para que as consultas sql posteriores
funcionem
'''
#Registrando o dataframe em uma view temporária
flights.createOrReplaceTempView("flights")

query = "FROM flights SELECT * LIMIT 10"

# Selecionando as 10 primeiras linhas do dataset
flights10 = spark.sql(query)

# Print o resultado
flights10.show()

+-----+--------+--------------------+-----------------+--------------------+-----------------+------------+----------------+------------------------------------------+--------------+----------------------------------------------+---------+----------------+------------------------------------+-----------------+--------------------+-------------------------+------------+------------+--------------+---------------------------------+--------------+--------------+--------------------------------+--------------------+------------------------------------+--------------------------+--------------------------------------+-------------------------+---------------------------------------------+------------------------+------------------------------------+--------------------------------+----------------------------+-----------------------------------+
|Idade|Demissão|  Viagem de negócios|Diárias recebidas|        Departamento|Distância de Casa|Escolaridade|Área de Formação|Número de contratos de 

In [ ]:
# Convertendo o resultado para pandas
flightspd = flights.toPandas()
display(flightspd)

,Idade,Demissão,Viagem de negócios,Diárias recebidas,Departamento,Distância de Casa,Escolaridade,Área de Formação,Número de contratos de trabalho na empresa,ID Funcionário,...,Satisfação nas relações Não trabalho,Jornada padrão de trabalho,opção de remaunração variável em ações,Total de anos trabalhados,Qtde de treinamentos realizados no último ano,Equilíbrio vida trabalho,Total de anos trabalhados na empresa,Anos trabalhados na função atual,Anos desde a última promoção,Anos trabalhando com o atual gestor
0,41,Sim,Viaja raramente,1102,Vendas,1,2,Ciências Humanas,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,Não,Viaja frequentemente,279,Pesquisa e Desenvolvimento,8,1,Ciências Humanas,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Sim,Viaja raramente,1373,Pesquisa e Desenvolvimento,2,2,Outras,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,Não,Viaja frequentemente,1392,Pesquisa e Desenvolvimento,3,4,Ciências Humanas,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,Não,Viaja raramente,591,Pesquisa e Desenvolvimento,2,1,Ciências Médicas,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Não,Viaja frequentemente,884,Pesquisa e Desenvolvimento,23,2,Ciências Médicas,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,Não,Viaja raramente,613,Pesquisa e Desenvolvimento,6,1,Ciências Médicas,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,Não,Viaja raramente,155,Pesquisa e Desenvolvimento,4,3,Ciências Humanas,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,Não,Viaja frequentemente,1023,Vendas,2,3,Ciências Médicas,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [ ]:
# Criando pandas dataframe
arq = "/content/funcionarios.csv"
pd_temp = pd.read_csv(arq)
display(pd_temp)

,Idade,Demissão,Viagem de negócios,Diárias recebidas,Departamento,Distância de Casa,Escolaridade,Área de Formação,Número de contratos de trabalho na empresa,ID Funcionário,...,Satisfação nas relações Não trabalho,Jornada padrão de trabalho,opção de remaunração variável em ações,Total de anos trabalhados,Qtde de treinamentos realizados no último ano,Equilíbrio vida trabalho,Total de anos trabalhados na empresa,Anos trabalhados na função atual,Anos desde a última promoção,Anos trabalhando com o atual gestor
0,41,Sim,Viaja raramente,1102,Vendas,1,2,Ciências Humanas,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,Não,Viaja frequentemente,279,Pesquisa e Desenvolvimento,8,1,Ciências Humanas,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Sim,Viaja raramente,1373,Pesquisa e Desenvolvimento,2,2,Outras,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,Não,Viaja frequentemente,1392,Pesquisa e Desenvolvimento,3,4,Ciências Humanas,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,Não,Viaja raramente,591,Pesquisa e Desenvolvimento,2,1,Ciências Médicas,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Não,Viaja frequentemente,884,Pesquisa e Desenvolvimento,23,2,Ciências Médicas,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,Não,Viaja raramente,613,Pesquisa e Desenvolvimento,6,1,Ciências Médicas,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,Não,Viaja raramente,155,Pesquisa e Desenvolvimento,4,3,Ciências Humanas,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,Não,Viaja frequentemente,1023,Vendas,2,3,Ciências Médicas,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [ ]:
# Cria spark_temp a partir de pd_temp
spark_temp = spark.createDataFrame(pd_temp.to_dict('records'))
spark_temp.show()

+----------------------------+--------------------------------+-----------------------------------+--------------------+--------+--------------------+-----------------+-----------------+------------------------+------------+------------+--------------+---------+--------------+-----+--------------------------+--------------+----------------------------------------------+------------------------------------+-----------------+------------------------------------------+---------------------------------------------+------------+--------------+----------------+-------------------------+------------------------------------+-------------------------+------------------------------------+--------------------+--------------------------------------+--------------------------------+---------------------------------+--------------------+----------------+
|Anos desde a última promoção|Anos trabalhados na função atual|Anos trabalhando com o atual gestor|               Cargo|Demissão|        Departamen

In [ ]:
data = {
    'nome': ['Ana', 'João', 'Maria', 'Ana', 'João', 'Maria', 'Ana'],
    'compras': [150, 200, 300, 100, 250, 400, 50]
}

dfpd = pd.DataFrame(data)
#Exibindo o DataFrame Original
print("DataFrame Original")
display(dfpd)

#Agrupando por nome e somando o total de compras por pessoa
dfpd_agrupado = dfpd.groupby('nome')['compras'].sum().reset_index()
#Exibindo o DataFrame Agrupado
print("DataFrame Agrupado")
display(dfpd_agrupado)



DataFrame Original


,nome,compras
0,Ana,150
1,João,200
2,Maria,300
3,Ana,100
4,João,250
5,Maria,400
6,Ana,50


DataFrame Agrupado


,compras
nome,
Ana,300
João,450
Maria,700
